<a href="https://colab.research.google.com/github/owenc8/IAT360_final_project/blob/main/final_proj_CV_IAT360.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>